## Installing Required Icetrait

In [82]:
!pip install git+https://github.com/vibhatha/pyiceberg_substrait@feat-schema-evolution-s1#egg=icetrait

  Cloning https://github.com/vibhatha/pyiceberg_substrait (to revision feat-schema-evolution-s1) to /tmp/pip-install-2em5yqze/icetrait_04802554c9014d07b436e87216df18f6
  Running command git clone --filter=blob:none --quiet https://github.com/vibhatha/pyiceberg_substrait /tmp/pip-install-2em5yqze/icetrait_04802554c9014d07b436e87216df18f6
  Running command git checkout -b feat-schema-evolution-s1 --track origin/feat-schema-evolution-s1
  Switched to a new branch 'feat-schema-evolution-s1'
  Branch 'feat-schema-evolution-s1' set up to track remote branch 'feat-schema-evolution-s1' from 'origin'.
  Resolved https://github.com/vibhatha/pyiceberg_substrait to commit 11f6c05bf12c7f2bf7c06d5b7a1b0845972a028f
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for icetrait: filename=icetrait-0.1

In [10]:
import os
os.environ['ICETRAIT_LOG_DIR'] = '/home/iceberg/notebooks/icetrait_logs'
os.getenv("ICETRAIT_LOG_DIR")

'/home/iceberg/notebooks/icetrait_logs'

**Warning**

Make sure to restart the kernel after installation

In [11]:
from icetrait.substrait.visitor import SubstraitPlanEditor, visit_and_update, RelVisitor, RelUpdateVisitor
from icetrait.duckdb.wrapper import DuckdbSubstrait
import duckdb

## Initialize Spark Environment

In [12]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.master("local")
    .appName("IcebergPySpark")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.demo.catalog-impl", "org.apache.iceberg.rest.RESTCatalog")
    .config("spark.sql.catalog.demo.uri", "http://rest:8181")
    .config("spark.sql.catalog.demo.s3.endpoint", "http://minio:9000")
    .config("spark.sql.defaultCatalog", "demo")
    .config("spark.eventLog.enabled", "true")
    .config("spark.eventLog.dir", "/home/iceberg/spark-events")
    .config("spark.history.fs.logDirectory", "/home/iceberg/spark-events")
    .config("spark.sql.catalogImplementation", "/home/iceberg/spark-events")
    .getOrCreate()
)
spark

23/05/25 05:20:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [13]:
%%sql

DROP TABLE IF EXISTS nyc_demo.taxis_sample;

23/05/25 05:20:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [14]:
%%sql

CREATE DATABASE IF NOT EXISTS nyc_demo;

23/05/25 05:20:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [15]:
%%sql

show databases;

23/05/25 05:20:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


namespace
nyc_demo


In [16]:
from pyiceberg.catalog import load_catalog

iceberg_catalog = load_catalog('default')
iceberg_catalog

In [58]:
iceberg_catalog.list_namespaces()

[('nyc_demo',)]

In [17]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-04.parquet")
df.write.saveAsTable("nyc_demo.taxis_sample")

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


## Simulating Iceberg and Icetrait on Simple Query

In [18]:
iceberg_table = iceberg_catalog.load_table("nyc_demo.taxis_sample")

In [19]:
iceberg_df = iceberg_table.scan().to_pandas()
head = iceberg_df.head()
head.dtypes

VendorID                               int64
tpep_pickup_datetime     datetime64[ns, UTC]
tpep_dropoff_datetime    datetime64[ns, UTC]
passenger_count                      float64
trip_distance                        float64
RatecodeID                           float64
store_and_fwd_flag                    object
PULocationID                           int64
DOLocationID                           int64
payment_type                           int64
fare_amount                          float64
extra                                float64
mta_tax                              float64
tip_amount                           float64
tolls_amount                         float64
improvement_surcharge                float64
total_amount                         float64
congestion_surcharge                 float64
airport_fee                          float64
dtype: object

In [21]:
"""
We expect the user to pass the evolved schema.
"""
## TODO: I think we can probably keep a single connection and use the update query to Spark applied on
## the duckdb connector and update the table.
def setup_duckdb():
    con = duckdb.connect()
    create_schema = "CREATE SCHEMA nyc_demo;"
    creation_query = """
    CREATE TABLE nyc_demo.taxis_sample (
        VendorID              bigint,
        tpep_pickup_datetime  timestamp,
        tpep_dropoff_datetime timestamp,
        passenger_count       double,
        trip_distance         double,
        RatecodeID            double,
        store_and_fwd_flag    string,
        PULocationID          bigint,
        DOLocationID          bigint,
        payment_type          bigint,
        fare_amount           double,
        extra                 double,
        mta_tax               double,
        tip_amount            double,
        tolls_amount          double,
        improvement_surcharge double,
        total_amount          double,
        congestion_surcharge  double,
        airport_fee           double
    );
    """
    con.execute(create_schema)
    con.execute(creation_query)
    return con

In [22]:
q1 = "SELECT * FROM nyc_demo.taxis_sample;"

In [23]:
!mkdir /home/iceberg/notebooks/s3

mkdir: cannot create directory ‘/home/iceberg/notebooks/s3’: File exists


In [24]:
wrapper = DuckdbSubstrait("default", "/home/iceberg/notebooks/s3", "nyc_demo", q1, setup_duckdb)
wrapper.update_named_table_with_schema()
wrapper.update_with_local_file_paths()

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/logging/__init__.py", line 1083, in emit
    msg = self.format(record)
  File "/usr/local/lib/python3.9/logging/__init__.py", line 927, in format
    return fmt.format(record)
  File "/usr/local/lib/python3.9/logging/__init__.py", line 663, in format
    record.message = record.getMessage()
  File "/usr/local/lib/python3.9/logging/__init__.py", line 367, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/local/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in la

In [25]:
duckdb_res = wrapper.execute()

In [26]:
df_duckdb_substrait = duckdb_res.to_df()
df_duckdb_substrait.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-04-01 00:00:18+00:00,2021-04-01 00:21:54+00:00,1.0,8.40,1.0,N,79,116,1,25.5,3.0,0.5,5.85,0.0,0.3,35.15,2.5,0.0
1,1,2021-04-01 00:42:37+00:00,2021-04-01 00:46:23+00:00,1.0,0.90,1.0,N,75,236,2,5.0,3.0,0.5,0.00,0.0,0.3,8.80,2.5,0.0
2,1,2021-04-01 00:57:56+00:00,2021-04-01 01:08:22+00:00,1.0,3.40,1.0,N,236,168,2,11.5,3.0,0.5,0.00,0.0,0.3,15.30,2.5,0.0
3,1,2021-04-01 00:01:58+00:00,2021-04-01 00:54:27+00:00,1.0,0.00,1.0,N,47,61,1,44.2,0.0,0.5,0.00,0.0,0.3,45.00,0.0,0.0
4,2,2021-04-01 00:24:55+00:00,2021-04-01 00:34:33+00:00,1.0,1.96,1.0,N,238,152,1,9.0,0.5,0.5,3.09,0.0,0.3,13.39,0.0,0.0


In [27]:
# TODO: create separate table per evaluating action so that we can do the validations properly
assert iceberg_df.equals(df_duckdb_substrait)

## Simulating Iceberg and Icetrait on Rename

In [28]:
%%sql

ALTER TABLE nyc_demo.taxis_sample RENAME COLUMN fare_amount TO fare

23/05/25 05:21:27 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/05/25 05:21:27 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


++
||
++
++

### Since the schema is updated, we need to update the Duckdb Substrait interface to reflect that change

In [29]:
# instead of `fare_amount` we use `fare` since it is the renamed field. 
def setup_duckdb_for_rename():
    con = duckdb.connect()
    create_schema = "CREATE SCHEMA nyc_demo;"
    creation_query = """
    CREATE TABLE nyc_demo.taxis_sample (
        VendorID              bigint,
        tpep_pickup_datetime  timestamp,
        tpep_dropoff_datetime timestamp,
        passenger_count       double,
        trip_distance         double,
        RatecodeID            double,
        store_and_fwd_flag    string,
        PULocationID          bigint,
        DOLocationID          bigint,
        payment_type          bigint,
        fare                  double,
        extra                 double,
        mta_tax               double,
        tip_amount            double,
        tolls_amount          double,
        improvement_surcharge double,
        total_amount          double,
        congestion_surcharge  double,
        airport_fee           double
    );
    """
    con.execute(create_schema)
    con.execute(creation_query)
    return con

In [30]:
query_rename = "SELECT fare FROM nyc_demo.taxis_sample;"

In [31]:
wrapper = DuckdbSubstrait("default", "/home/iceberg/notebooks/s3", "nyc_demo", query_rename, setup_duckdb_for_rename)
wrapper.update_named_table_with_schema()
wrapper.update_with_local_file_paths()

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/logging/__init__.py", line 1083, in emit
    msg = self.format(record)
  File "/usr/local/lib/python3.9/logging/__init__.py", line 927, in format
    return fmt.format(record)
  File "/usr/local/lib/python3.9/logging/__init__.py", line 663, in format
    record.message = record.getMessage()
  File "/usr/local/lib/python3.9/logging/__init__.py", line 367, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/local/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in la

In [32]:
wrapper.plan

relations {
  root {
    input {
      project {
        input {
          read {
            base_schema {
              names: "VendorID"
              names: "tpep_pickup_datetime"
              names: "tpep_dropoff_datetime"
              names: "passenger_count"
              names: "trip_distance"
              names: "RatecodeID"
              names: "store_and_fwd_flag"
              names: "PULocationID"
              names: "DOLocationID"
              names: "payment_type"
              names: "fare"
              names: "extra"
              names: "mta_tax"
              names: "tip_amount"
              names: "tolls_amount"
              names: "improvement_surcharge"
              names: "total_amount"
              names: "congestion_surcharge"
              names: "airport_fee"
              struct {
                types {
                  i64 {
                    nullability: NULLABILITY_NULLABLE
                  }
                }
                types {
      

In [33]:
wrapper.updated_plan

relations {
  root {
    input {
      project {
        input {
          read {
            base_schema {
              names: "VendorID"
              names: "tpep_pickup_datetime"
              names: "tpep_dropoff_datetime"
              names: "passenger_count"
              names: "trip_distance"
              names: "RatecodeID"
              names: "store_and_fwd_flag"
              names: "PULocationID"
              names: "DOLocationID"
              names: "payment_type"
              names: "fare_amount"
              names: "extra"
              names: "mta_tax"
              names: "tip_amount"
              names: "tolls_amount"
              names: "improvement_surcharge"
              names: "total_amount"
              names: "congestion_surcharge"
              names: "airport_fee"
              struct {
                types {
                  i64 {
                    nullability: NULLABILITY_NULLABLE
                  }
                }
                types {

In [34]:
duckdb_rename_res = wrapper.execute()

In [35]:
df_duckdb_substrait_rename = duckdb_rename_res.to_df()
df_duckdb_substrait_rename.head()

,fare
0,25.5
1,5.0
2,11.5
3,44.2
4,9.0


In [36]:
iceberg_catalog = load_catalog('default')
iceberg_table = iceberg_catalog.load_table("nyc_demo.taxis_sample")
iceberg_df_rename = iceberg_table.scan(selected_fields=["fare"]).to_pandas()
head_rename = iceberg_df_rename.head()
head_rename

,fare
0,25.5
1,5.0
2,11.5
3,44.2
4,9.0


In [37]:
assert iceberg_df_rename.equals(df_duckdb_substrait_rename)

## Simulating Iceberg and Icetrait on Add

In [38]:
%%sql

ALTER TABLE nyc_demo.taxis_sample
ADD COLUMN fare_per_distance_unit float AFTER trip_distance

23/05/25 05:21:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/05/25 05:21:51 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


++
||
++
++

In [39]:
%%sql

UPDATE nyc_demo.taxis_sample
SET fare_per_distance_unit = fare/trip_distance

23/05/25 05:21:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

### Since the schema is updated, we need to update the Duckdb Substrait interface to reflect that change

In [40]:
# adding field `fare_per_distance_unit`
def setup_duckdb_for_update():
    con = duckdb.connect()
    create_schema = "CREATE SCHEMA nyc_demo;"
    creation_query = """
    CREATE TABLE nyc_demo.taxis_sample (
        VendorID               bigint,
        tpep_pickup_datetime   timestamp,
        tpep_dropoff_datetime  timestamp,
        passenger_count        double,
        trip_distance          double,
        fare_per_distance_unit float, 
        RatecodeID             double,
        store_and_fwd_flag     string,
        PULocationID           bigint,
        DOLocationID           bigint,
        payment_type           bigint,
        fare                   double,
        extra                  double,
        mta_tax                double,
        tip_amount             double,
        tolls_amount           double,
        improvement_surcharge  double,
        total_amount           double,
        congestion_surcharge   double,
        airport_fee            double
    );
    """
    con.execute(create_schema)
    con.execute(creation_query)
    return con

In [41]:
query_add = "SELECT fare_per_distance_unit, fare FROM nyc_demo.taxis_sample;"

In [42]:
wrapper = DuckdbSubstrait("default", "/home/iceberg/notebooks/s3", "nyc_demo", query_add, setup_duckdb_for_update)
wrapper.update_named_table_with_schema()
wrapper.update_with_local_file_paths()

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/logging/__init__.py", line 1083, in emit
    msg = self.format(record)
  File "/usr/local/lib/python3.9/logging/__init__.py", line 927, in format
    return fmt.format(record)
  File "/usr/local/lib/python3.9/logging/__init__.py", line 663, in format
    record.message = record.getMessage()
  File "/usr/local/lib/python3.9/logging/__init__.py", line 367, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/local/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in la

In [43]:
duckdb_add_res = wrapper.execute()

In [44]:
df_duckdb_substrait_add = duckdb_add_res.to_df()
df_duckdb_substrait_add.head()

,fare_per_distance_unit,fare
0,3.035714,25.5
1,5.555555,5.0
2,3.382353,11.5
3,NaN,44.2
4,4.591837,9.0


In [45]:
iceberg_catalog = load_catalog('default')
iceberg_table = iceberg_catalog.load_table("nyc_demo.taxis_sample")
iceberg_df_add = iceberg_table.scan(selected_fields=["fare_per_distance_unit", "fare"]).to_pandas()
head_add = iceberg_df_add.head()
head_add

,fare_per_distance_unit,fare
0,3.035714,25.5
1,5.555555,5.0
2,3.382353,11.5
3,NaN,44.2
4,4.591837,9.0


In [46]:
assert iceberg_df_add.equals(df_duckdb_substrait_add)

## Simulating Iceberg and Icetrait on Drop

In [47]:
%%sql
ALTER TABLE nyc_demo.taxis_sample
DROP COLUMN improvement_surcharge;

23/05/25 05:22:16 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/05/25 05:22:16 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


++
||
++
++

### Since the schema is updated, we need to update the Duckdb Substrait interface to reflect that change

In [48]:
# field `improvement_surcharge` is included duckdb assumes the fullschema to be available and
# it handles selection via projection
def setup_duckdb_for_drop():
    con = duckdb.connect()
    create_schema = "CREATE SCHEMA nyc_demo;"
    creation_query = """
    CREATE TABLE nyc_demo.taxis_sample (
        VendorID               bigint,
        tpep_pickup_datetime   timestamp,
        tpep_dropoff_datetime  timestamp,
        passenger_count        double,
        trip_distance          double,
        fare_per_distance_unit float, 
        RatecodeID             double,
        store_and_fwd_flag     string,
        PULocationID           bigint,
        DOLocationID           bigint,
        payment_type           bigint,
        fare                   double,
        extra                  double,
        mta_tax                double,
        tip_amount             double,
        tolls_amount           double,
        total_amount           double,
        congestion_surcharge   double,
        airport_fee            double
    );
    """
    con.execute(create_schema)
    con.execute(creation_query)
    return con

In [49]:
query_drop = "SELECT * FROM nyc_demo.taxis_sample;"

In [50]:
wrapper = DuckdbSubstrait("default", "/home/iceberg/notebooks/s3", "nyc_demo", query_drop, setup_duckdb_for_drop)
wrapper.update_named_table_with_schema()
wrapper.update_with_local_file_paths()

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/logging/__init__.py", line 1083, in emit
    msg = self.format(record)
  File "/usr/local/lib/python3.9/logging/__init__.py", line 927, in format
    return fmt.format(record)
  File "/usr/local/lib/python3.9/logging/__init__.py", line 663, in format
    record.message = record.getMessage()
  File "/usr/local/lib/python3.9/logging/__init__.py", line 367, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/local/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in la

In [51]:
wrapper.plan

relations {
  root {
    input {
      project {
        input {
          read {
            base_schema {
              names: "VendorID"
              names: "tpep_pickup_datetime"
              names: "tpep_dropoff_datetime"
              names: "passenger_count"
              names: "trip_distance"
              names: "fare_per_distance_unit"
              names: "RatecodeID"
              names: "store_and_fwd_flag"
              names: "PULocationID"
              names: "DOLocationID"
              names: "payment_type"
              names: "fare"
              names: "extra"
              names: "mta_tax"
              names: "tip_amount"
              names: "tolls_amount"
              names: "total_amount"
              names: "congestion_surcharge"
              names: "airport_fee"
              struct {
                types {
                  i64 {
                    nullability: NULLABILITY_NULLABLE
                  }
                }
                types {
     

In [53]:
wrapper.updated_plan

relations {
  root {
    input {
      project {
        input {
          read {
            base_schema {
              names: "VendorID"
              names: "tpep_pickup_datetime"
              names: "tpep_dropoff_datetime"
              names: "passenger_count"
              names: "trip_distance"
              names: "fare_per_distance_unit"
              names: "RatecodeID"
              names: "store_and_fwd_flag"
              names: "PULocationID"
              names: "DOLocationID"
              names: "payment_type"
              names: "fare"
              names: "extra"
              names: "mta_tax"
              names: "tip_amount"
              names: "tolls_amount"
              names: "improvement_surcharge"
              names: "total_amount"
              names: "congestion_surcharge"
              names: "airport_fee"
              struct {
                types {
                  i64 {
                    nullability: NULLABILITY_NULLABLE
                  }
  

In [54]:
duckdb_drop_res = wrapper.execute()

In [55]:
df_duckdb_substrait_drop = duckdb_drop_res.to_df()
df_duckdb_substrait_drop.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,fare_per_distance_unit,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare,extra,mta_tax,tip_amount,tolls_amount,total_amount,congestion_surcharge,airport_fee
0,1,2021-04-01 00:00:18+00:00,2021-04-01 00:21:54+00:00,1.0,8.40,3.035714,1.0,N,79,116,1,25.5,3.0,0.5,5.85,0.0,35.15,2.5,0.0
1,1,2021-04-01 00:42:37+00:00,2021-04-01 00:46:23+00:00,1.0,0.90,5.555555,1.0,N,75,236,2,5.0,3.0,0.5,0.00,0.0,8.80,2.5,0.0
2,1,2021-04-01 00:57:56+00:00,2021-04-01 01:08:22+00:00,1.0,3.40,3.382353,1.0,N,236,168,2,11.5,3.0,0.5,0.00,0.0,15.30,2.5,0.0
3,1,2021-04-01 00:01:58+00:00,2021-04-01 00:54:27+00:00,1.0,0.00,NaN,1.0,N,47,61,1,44.2,0.0,0.5,0.00,0.0,45.00,0.0,0.0
4,2,2021-04-01 00:24:55+00:00,2021-04-01 00:34:33+00:00,1.0,1.96,4.591837,1.0,N,238,152,1,9.0,0.5,0.5,3.09,0.0,13.39,0.0,0.0


In [56]:
iceberg_catalog = load_catalog('default')
iceberg_table = iceberg_catalog.load_table("nyc_demo.taxis_sample")

In [57]:
iceberg_drop_df = iceberg_table.scan().to_pandas()
iceberg_drop_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,fare_per_distance_unit,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-04-01 00:00:18+00:00,2021-04-01 00:21:54+00:00,1.0,8.40,3.035714,1.0,N,79,116,1,25.5,3.0,0.5,5.85,0.0,0.3,35.15,2.5,0.0
1,1,2021-04-01 00:42:37+00:00,2021-04-01 00:46:23+00:00,1.0,0.90,5.555555,1.0,N,75,236,2,5.0,3.0,0.5,0.00,0.0,0.3,8.80,2.5,0.0
2,1,2021-04-01 00:57:56+00:00,2021-04-01 01:08:22+00:00,1.0,3.40,3.382353,1.0,N,236,168,2,11.5,3.0,0.5,0.00,0.0,0.3,15.30,2.5,0.0
3,1,2021-04-01 00:01:58+00:00,2021-04-01 00:54:27+00:00,1.0,0.00,NaN,1.0,N,47,61,1,44.2,0.0,0.5,0.00,0.0,0.3,45.00,0.0,0.0
4,2,2021-04-01 00:24:55+00:00,2021-04-01 00:34:33+00:00,1.0,1.96,4.591837,1.0,N,238,152,1,9.0,0.5,0.5,3.09,0.0,0.3,13.39,0.0,0.0


## Drop Only Simulation 

In [58]:
%%sql

DROP TABLE IF EXISTS nyc_demo.taxis_sample;

23/05/25 05:22:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [59]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-04.parquet")
df.write.saveAsTable("nyc_demo.taxis_sample")

In [60]:
%%sql
ALTER TABLE nyc_demo.taxis_sample
DROP COLUMN improvement_surcharge;

23/05/25 05:22:48 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/05/25 05:22:48 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


++
||
++
++

In [61]:
%%sql
SELECT * FROM nyc_demo.taxis_sample LIMIT 5;

23/05/25 05:22:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount,congestion_surcharge,airport_fee
1,2021-04-01 00:00:18,2021-04-01 00:21:54,1.0,8.4,1.0,N,79,116,1,25.5,3.0,0.5,5.85,0.0,35.15,2.5,0.0
1,2021-04-01 00:42:37,2021-04-01 00:46:23,1.0,0.9,1.0,N,75,236,2,5.0,3.0,0.5,0.0,0.0,8.8,2.5,0.0
1,2021-04-01 00:57:56,2021-04-01 01:08:22,1.0,3.4,1.0,N,236,168,2,11.5,3.0,0.5,0.0,0.0,15.3,2.5,0.0
1,2021-04-01 00:01:58,2021-04-01 00:54:27,1.0,0.0,1.0,N,47,61,1,44.2,0.0,0.5,0.0,0.0,45.0,0.0,0.0
2,2021-04-01 00:24:55,2021-04-01 00:34:33,1.0,1.96,1.0,N,238,152,1,9.0,0.5,0.5,3.09,0.0,13.39,0.0,0.0


In [62]:
iceberg_catalog = load_catalog('default')
iceberg_table_drop_s = iceberg_catalog.load_table("nyc_demo.taxis_sample")

In [63]:
iceberg_drop_s_df = iceberg_table_drop_s.scan().to_pandas()
iceberg_drop_s_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount,congestion_surcharge,airport_fee
0,1,2021-04-01 00:00:18+00:00,2021-04-01 00:21:54+00:00,1.0,8.40,1.0,N,79,116,1,25.5,3.0,0.5,5.85,0.0,35.15,2.5,0.0
1,1,2021-04-01 00:42:37+00:00,2021-04-01 00:46:23+00:00,1.0,0.90,1.0,N,75,236,2,5.0,3.0,0.5,0.00,0.0,8.80,2.5,0.0
2,1,2021-04-01 00:57:56+00:00,2021-04-01 01:08:22+00:00,1.0,3.40,1.0,N,236,168,2,11.5,3.0,0.5,0.00,0.0,15.30,2.5,0.0
3,1,2021-04-01 00:01:58+00:00,2021-04-01 00:54:27+00:00,1.0,0.00,1.0,N,47,61,1,44.2,0.0,0.5,0.00,0.0,45.00,0.0,0.0
4,2,2021-04-01 00:24:55+00:00,2021-04-01 00:34:33+00:00,1.0,1.96,1.0,N,238,152,1,9.0,0.5,0.5,3.09,0.0,13.39,0.0,0.0


In [64]:
def setup_duckdb_for_standalone_drop():
    con = duckdb.connect()
    create_schema = "CREATE SCHEMA nyc_demo;"
    creation_query = """
    CREATE TABLE nyc_demo.taxis_sample (
        VendorID               bigint,
        tpep_pickup_datetime   timestamp,
        tpep_dropoff_datetime  timestamp,
        passenger_count        double,
        trip_distance          double,
        RatecodeID             double,
        store_and_fwd_flag     string,
        PULocationID           bigint,
        DOLocationID           bigint,
        payment_type           bigint,
        fare_amount            double,
        extra                  double,
        mta_tax                double,
        tip_amount             double,
        tolls_amount           double,
        total_amount           double,
        congestion_surcharge   double,
        airport_fee            double
    );
    """
    con.execute(create_schema)
    con.execute(creation_query)
    return con

In [65]:
query_s_drop = "SELECT * FROM nyc_demo.taxis_sample;"

In [66]:
wrapper = DuckdbSubstrait("default", "/home/iceberg/notebooks/s3", "nyc_demo", query_s_drop, setup_duckdb_for_standalone_drop)
wrapper.update_named_table_with_schema()
wrapper.update_with_local_file_paths()

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/logging/__init__.py", line 1083, in emit
    msg = self.format(record)
  File "/usr/local/lib/python3.9/logging/__init__.py", line 927, in format
    return fmt.format(record)
  File "/usr/local/lib/python3.9/logging/__init__.py", line 663, in format
    record.message = record.getMessage()
  File "/usr/local/lib/python3.9/logging/__init__.py", line 367, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/local/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in la

In [67]:
duckdb_drop_s_res = wrapper.execute()

In [68]:
df_duckdb_substrait_s_drop = duckdb_drop_s_res.to_df()
df_duckdb_substrait_s_drop.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount,congestion_surcharge,airport_fee
0,1,2021-04-01 00:00:18+00:00,2021-04-01 00:21:54+00:00,1.0,8.40,1.0,N,79,116,1,25.5,3.0,0.5,5.85,0.0,35.15,2.5,0.0
1,1,2021-04-01 00:42:37+00:00,2021-04-01 00:46:23+00:00,1.0,0.90,1.0,N,75,236,2,5.0,3.0,0.5,0.00,0.0,8.80,2.5,0.0
2,1,2021-04-01 00:57:56+00:00,2021-04-01 01:08:22+00:00,1.0,3.40,1.0,N,236,168,2,11.5,3.0,0.5,0.00,0.0,15.30,2.5,0.0
3,1,2021-04-01 00:01:58+00:00,2021-04-01 00:54:27+00:00,1.0,0.00,1.0,N,47,61,1,44.2,0.0,0.5,0.00,0.0,45.00,0.0,0.0
4,2,2021-04-01 00:24:55+00:00,2021-04-01 00:34:33+00:00,1.0,1.96,1.0,N,238,152,1,9.0,0.5,0.5,3.09,0.0,13.39,0.0,0.0


In [69]:
assert iceberg_drop_s_df.equals(df_duckdb_substrait_s_drop)